In [ ]:
# Compute the LP solution for the first instance
import os
import sys
import pickle
import numpy as np
# Get the absolute path of the src directory
current_dir = os.getcwd()
src_path = os.path.join(current_dir, '../src')
sys.path.append(src_path)
# Load the dataset from the file
import experiments,inamdar_alg
import importlib
import setcover_dataset,setcover_lp
importlib.reload(experiments)
importlib.reload(inamdar_alg)

from experiments import read_dataset
from experiments import run_experiments
from setcover_dataset import generate_setcover_dataset


In [ ]:
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import importlib
import experiments
importlib.reload(experiments)



from experiments import read_dataset, write_dataset, run_experiments
def run_experiment_for_f(f_value):
    """
    Run experiment for a specific f value
    """
    print(f"Starting experiment for f = {f_value}")
    start_time = time.time()
    
    try:
        # Load dataset for this f value
        dataset_path = f'../data/freq_data/final_dataset/instances/setcover_small_dataset_f_{f_value}.pkl'
        loaded_dataset = read_dataset(dataset_path)
        
        # Run experiments for this f value
        results = run_experiments(loaded_dataset, f_value)
        path = f'../data/freq_data/final_dataset/results/results_{f_value}.pkl'
        write_dataset(path, results)
        end_time = time.time()
        print(f"Completed experiment for f = {f_value} in {end_time - start_time:.2f} seconds")
        
        return f_value, results, True
        
    except Exception as e:
        print(f"Error in experiment for f = {f_value}: {str(e)}")
        return f_value, None, False

def run_parallel_experiments(f_values, max_workers=4):
    """
    Run experiments for multiple f values in parallel
    """
    results_dict = {}
    
    print(f"Starting parallel experiments for f values: {f_values}")
    print(f"Using {max_workers} worker threads")
    
    overall_start_time = time.time()
    
    # Use ThreadPoolExecutor to run experiments in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all experiments
        future_to_f = {executor.submit(run_experiment_for_f, f): f for f in f_values}
        
        # Collect results as they complete
        for future in as_completed(future_to_f):
            f_value = future_to_f[future]
            try:
                f, results, success = future.result()
                if success:
                    results_dict[f] = results
                    print(f"✓ Successfully stored results for f = {f}")
                else:
                    print(f"✗ Failed to get results for f = {f}")
            except Exception as e:
                print(f"✗ Exception occurred for f = {f_value}: {str(e)}")
    
    overall_end_time = time.time()
    print(f"\nAll experiments completed in {overall_end_time - overall_start_time:.2f} seconds")
    print(f"Successfully completed {len(results_dict)} out of {len(f_values)} experiments")
    
    return results_dict

# Example usage: Run experiments for multiple f values in parallel
f_values = [3, 4, 5, 6, 7, 8, 9, 10]
max_workers = 4  # Adjust based on your system's capabilities

# Run parallel experiments
all_results = run_parallel_experiments(f_values, max_workers=max_workers)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

sns.set(style="whitegrid")
sns.set_palette("Set2") 

positions_fair = [i*2-0.4 for i in range(1, len(f_values)+1)]
positions_inamdar = [i*2+0.4 for i in range(1, len(f_values)+1)]

plt.figure(figsize=(10, 6))
bp1 = plt.boxplot(fair_iter_times, positions=positions_fair, widths=0.6, patch_artist=True, boxprops=dict(facecolor='#66c2a5'), labels=f_values)
bp2 = plt.boxplot(inamdar_times, positions=positions_inamdar, widths=0.6, patch_artist=True, boxprops=dict(facecolor='#fc8d62'))

plt.xticks([i*2 for i in range(1, len(f_values)+1)], f_values)
plt.xlabel('Frequency (f)')
plt.ylabel('Running Time (seconds)')
plt.title('Running Time Comparison: fair_iter vs inamdar')

# Custom legend
legend_handles = [
    Patch(facecolor='#66c2a5', label='fair_iter'),
    Patch(facecolor='#fc8d62', label='inamdar')
]
plt.legend(handles=legend_handles)

plt.grid(axis='y')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

f_values = [3, 4, 5, 6, 7, 8, 9, 10]
# Prepare data for boxplots
fair_iter_approx = [all_results[f]['approximation_ratio_fair_iter'] if f!=3 else all_results[3][1]['approximation_ratio_fair_iter'] for f in f_values]
inamdar_approx = [all_results[f]['approximation_ratio_inamdar'] if f!=3 else all_results[3][1]['approximation_ratio_inamdar'] for f in f_values]
positions_fair = [i*2-0.4 for i in range(1, len(f_values)+1)]
positions_inamdar = [i*2+0.4 for i in range(1, len(f_values)+1)]

bp1 = plt.boxplot(fair_iter_approx, positions=positions_fair, widths=0.6, patch_artist=True, boxprops=dict(facecolor='#66c2a5'), labels=f_values)
bp2 = plt.boxplot(inamdar_approx, positions=positions_inamdar, widths=0.6, patch_artist=True, boxprops=dict(facecolor='#fc8d62'))

plt.xticks([i*2 for i in range(1, len(f_values)+1)], f_values)
plt.xlabel('Frequency (f)')
plt.ylabel('Approximation Ratio')
plt.title('Approximation Ratio Comparison: fair_iter vs inamdar')

plt.legend(handles=legend_handles)
plt.grid(axis='y')
plt.show()

In [ ]:
loaded_dataset = read_dataset('../data/freq_data/final_dataset/instances/setcover_small_dataset_f_3.pkl')
print(max( list(loaded_dataset[0]['element_frequencies'].values()) ))


In [ ]:
import seaborn as sns

# Compute worst-case and best-case (min) approximation ratios for shading
worst_fair_iter = [max(ratios) for ratios in fair_iter_approx]
best_fair_iter = [min(ratios) for ratios in fair_iter_approx]
worst_inamdar = [max(ratios) for ratios in inamdar_approx]
best_inamdar = [min(ratios) for ratios in inamdar_approx]
mean_fair_iter = [np.mean(ratios) for ratios in fair_iter_approx]
mean_inamdar = [np.mean(ratios) for ratios in inamdar_approx]

plt.figure(figsize=(10, 6))
sns.lineplot(x=f_values, y=worst_fair_iter, marker='o', label='Fair Iterated Rounding (worst case)', color='limegreen')
sns.lineplot(x=f_values, y=worst_inamdar, marker='o', label='Inamdar Rounding (worst case)', color='red')
sns.lineplot(x=f_values, y=mean_fair_iter, marker='o', label='Fair Iterated Rounding (mean)', color='darkgreen')
sns.lineplot(x=f_values, y=mean_inamdar, marker='o', label='Inamdar Rounding (mean)', color='darkred')
# Shaded region for fair_iter
plt.fill_between(f_values, best_fair_iter, worst_fair_iter, color='limegreen', alpha=0.2)
# Shaded region for inamdar
plt.fill_between(f_values, best_inamdar, worst_inamdar, color='orange', alpha=0.2)

plt.xlabel('Frequency (f)')
plt.ylabel('Worst-case Approximation Ratio')
plt.title('Worst-case Approximation Ratio vs Frequency')
plt.legend()
plt.grid(True)
plt.savefig('../data/freq_data/final_dataset/plots/worst_case_approximation_ratios_vs_f.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
# Compute average number of elements satisfying coverage requirements for each f
mean_fair_iter_coverage = [np.mean(cov) for cov in fair_iter_coverage]
mean_inamdar_coverage = [np.mean(cov) for cov in inamdar_coverage]

# Compute min and max for shading
min_fair_iter_coverage = [np.min(cov) for cov in fair_iter_coverage]
max_fair_iter_coverage = [np.max(cov) for cov in fair_iter_coverage]
min_inamdar_coverage = [np.min(cov) for cov in inamdar_coverage]
max_inamdar_coverage = [np.max(cov) for cov in inamdar_coverage]

plt.plot(f_values, mean_fair_iter_coverage, marker='o', label='Fair Iterated Rounding', color='#66c2a5')
plt.plot(f_values, mean_inamdar_coverage, marker='o', label='Inamdar Rounding', color='#fc8d62')

plt.fill_between(f_values, min_fair_iter_coverage, max_fair_iter_coverage, color='#66c2a5', alpha=0.2)
plt.fill_between(f_values, min_inamdar_coverage, max_inamdar_coverage, color='#fc8d62', alpha=0.2)

plt.xlabel('Frequency (f)')
plt.ylabel('Number of Elements Satisfying Coverage Requirements')
plt.title('Coverage Satisfaction vs Frequency')
plt.legend()
plt.grid(axis='y')
plt.savefig('../data/freq_data/final_dataset/plots/coverage_satisfaction_vs_f.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
all_results[3][1].keys()

In [ ]:
plt.figure(figsize=(10, 6))

# Compute mean objectives for each f value
mean_fair_iter_obj = [np.mean(obj) for obj in [all_results[f]['tim'] if f != 3 else all_results[3][1]['fair_iter_objectives'] for f in f_values]]
mean_inamdar_obj = [np.mean(obj) for obj in [all_results[f]['inamdar_objectives'] if f != 3 else all_results[3][1]['inamdar_objectives'] for f in f_values]]
# mean_lp_obj = [np.mean(obj) for obj in [all_results[f]['lp_objectives'] if f != 3 else all_results[3][1]['lp_objectives'] for f in f_values]]

# Compute min and max for shading
min_fair_iter_obj = [np.min(obj) for obj in [all_results[f]['fair_iter_objectives'] if f != 3 else all_results[3][1]['fair_iter_objectives'] for f in f_values]]
max_fair_iter_obj = [np.max(obj) for obj in [all_results[f]['fair_iter_objectives'] if f != 3 else all_results[3][1]['fair_iter_objectives'] for f in f_values]]
min_inamdar_obj = [np.min(obj) for obj in [all_results[f]['inamdar_objectives'] if f != 3 else all_results[3][1]['inamdar_objectives'] for f in f_values]]
max_inamdar_obj = [np.max(obj) for obj in [all_results[f]['inamdar_objectives'] if f != 3 else all_results[3][1]['inamdar_objectives'] for f in f_values]]

plt.plot(f_values, mean_fair_iter_obj, marker='o', label='Fair Iterated Rounding', color='#66c2a5')
plt.plot(f_values, mean_inamdar_obj, marker='o', label='Inamdar Rounding', color='#fc8d62')
plt.plot(f_values, mean_lp_obj, marker='o', label='LP Relaxation', color='#8da0cb')

plt.fill_between(f_values, min_fair_iter_obj, max_fair_iter_obj, color='#66c2a5', alpha=0.2)
plt.fill_between(f_values, min_inamdar_obj, max_inamdar_obj, color='#fc8d62', alpha=0.2)

plt.xlabel('Max Frequency (f)')
plt.ylabel('Mean Objective Value')
plt.title('Mean Objective Value vs Frequency')
plt.legend()
plt.grid(axis='y')
plt.savefig('../data/freq_data/final_dataset/plots/mean_objectives_vs_f.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f_values, mean_fair_iter_time, marker='o', label='Fair Iterated Rounding', color='#66c2a5')
plt.plot(f_values, mean_inamdar_time, marker='o', label='Inamdar Rounding', color='#fc8d62')

plt.fill_between(f_values, min_fair_iter_time, max_fair_iter_time, color='#66c2a5', alpha=0.2)
plt.fill_between(f_values, min_inamdar_time, max_inamdar_time, color='#fc8d62', alpha=0.2)

plt.xlabel('Frequency (f)')
plt.ylabel('Running Time (seconds) for 100 instances')
plt.title('Running Time Comparison: Fair Iterated Rounding vs Inamdar Rounding')
plt.legend(handles=legend_handles)
plt.grid(axis='y')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))

# Compute mean objectives for each f value
mean_fair_iter_obj = [np.mean(obj) for obj in [all_results[f]['times_fair_iter'] if f != 3 else all_results[3][1]['times_inamdar'] for f in f_values]]
mean_inamdar_obj = [np.mean(obj) for obj in [all_results[f]['times_inamdar'] if f != 3 else all_results[3][1]['times_inamdar'] for f in f_values]]
# mean_lp_obj = [np.mean(obj) for obj in [all_results[f]['lp_objectives'] if f != 3 else all_results[3][1]['lp_objectives'] for f in f_values]]

# Compute min and max for shading
min_fair_iter_obj = [np.min(obj) for obj in [all_results[f]['fair_iter_objectives'] if f != 3 else all_results[3][1]['fair_iter_objectives'] for f in f_values]]
max_fair_iter_obj = [np.max(obj) for obj in [all_results[f]['fair_iter_objectives'] if f != 3 else all_results[3][1]['fair_iter_objectives'] for f in f_values]]
min_inamdar_obj = [np.min(obj) for obj in [all_results[f]['inamdar_objectives'] if f != 3 else all_results[3][1]['inamdar_objectives'] for f in f_values]]
max_inamdar_obj = [np.max(obj) for obj in [all_results[f]['inamdar_objectives'] if f != 3 else all_results[3][1]['inamdar_objectives'] for f in f_values]]

plt.plot(f_values, mean_fair_iter_obj, marker='o', label='Fair Iterated Rounding', color='#66c2a5')
plt.plot(f_values, mean_inamdar_obj, marker='o', label='Inamdar Rounding', color='#fc8d62')
# plt.plot(f_values, mean_lp_obj, marker='o', label='LP Relaxation', color='#8da0cb')

plt.fill_between(f_values, min_fair_iter_obj, max_fair_iter_obj, color='#66c2a5', alpha=0.2)
plt.fill_between(f_values, min_inamdar_obj, max_inamdar_obj, color='#fc8d62', alpha=0.2)

plt.xlabel('Max Frequency (f)')
plt.ylabel('Mean Running Time (seconds) for 100 Instances')
plt.title('Mean Running Time vs Frequency')
plt.legend()
plt.grid(axis='y')
plt.savefig('../data/freq_data/final_dataset/plots/mean_objectives_vs_f.png', dpi=300, bbox_inches='tight')
plt.show()
